In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
base_model = 'bigdefence/Llama-3.1-8B-bigdefence-Ko'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="auto")
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
from transformers import TextStreamer
def generate_response(prompt, model, tokenizer, text_streamer,max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    inputs = inputs.to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            streamer=text_streamer,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(prompt, '').strip()



In [ ]:
key = "대한민국 대통령은?"
prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{key}

### Response:
"""
text_streamer = TextStreamer(tokenizer)
response = generate_response(prompt, model, tokenizer,text_streamer)
print(response)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
대한민국 대통령은?

### Response:
대한민국의 대통령은 대한민국의 국가원수이자 행정권을 가진 국가의 정상이다. 1948년 8월 15일 대한민국 정부가 수립되면서 대통령은 대한민국 정부의 최고 권력자로 지정되었다. 대한민국 정부 수립 이후 대통령은 1963년 3월 12일 제3공화국 시절 김영삼, 1987년 2월 25일 제6공화국 시절 김영삼, 1993년 2월 25일 제4공화국 시절 김영삼, 1998년 2월 25일 제5공화국 시절 김대중, 2003년 2월 25일 제6공화국 시절 노무현, 2008년 2월 25일 제7공화국 시절 이명박, 2013년 2월 25일 제8공화국 시절 박근혜, 2017년 5월 10일 제9공화국 시절 문재인, 2022년 5월 10일 제10공화국 시절 윤석열이 역임하였다.<|end_of_text|>
대한민국의 대통령은 대한민국의 국가원수이자 행정권을 가진 국가의 정상이다. 1948년 8월 15일 대한민국 정부가 수립되면서 대통령은 대한민국 정부의 최고 권력자로 지정되었다. 대한민국 정부 수립 이후 대통령은 1963년 3월 12일 제3공화국 시절 김영삼, 1987년 2월 25일 제6공화국 시절 김영삼, 1993년 2월 25일 제4공화국 시절 김영삼, 1998년 2월 25일 제5공화국 시절 김대중, 2003년 2월 25일 제6공화국 시절 노무현, 2008년 2월 25일 제7공화국 시절 이명박, 2013년 2월 25일 제8공화국 시절 박근혜, 2017년 5월 10일 제9공화국 시절 문재인, 2022년 5월 10일 제10공화국 시절 윤석열이 역임하였다.
